<a href="https://colab.research.google.com/github/J1406/myproyet_/blob/main/Objects_DetectionJM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install torch torchvision requests validators transformers qrcode gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 698.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install gradio -q

In [8]:
import io
import gradio as gr
import matplotlib.pyplot as plt
import requests, validators
import torch
import pathlib
from PIL import Image
from transformers import AutoFeatureExtractor, DetrForObjectDetection, YolosForObjectDetection

import os

# colors for visualization
COLORS = [
    [0.000, 0.447, 0.741],
    [0.850, 0.325, 0.098],
    [0.929, 0.694, 0.125],
    [0.494, 0.184, 0.556],
    [0.466, 0.674, 0.188],
    [0.301, 0.745, 0.933]
]

def make_prediction(img, feature_extractor, model):
    inputs = feature_extractor(img, return_tensors="pt")
    outputs = model(**inputs)
    img_size = torch.tensor([tuple(reversed(img.size))])
    processed_outputs = feature_extractor.post_process(outputs, img_size)
    return processed_outputs[0]

def fig2img(fig):
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img


def visualize_prediction(pil_img, output_dict, threshold=0.7, id2label=None):
    keep = output_dict["scores"] > threshold
    boxes = output_dict["boxes"][keep].tolist()
    scores = output_dict["scores"][keep].tolist()
    labels = output_dict["labels"][keep].tolist()
    if id2label is not None:
        labels = [id2label[x] for x in labels]

    plt.figure(figsize=(16, 10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, (xmin, ymin, xmax, ymax), label, color in zip(scores, boxes, labels, colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=color, linewidth=3))
        ax.text(xmin, ymin, f"{label}: {score:0.2f}", fontsize=15, bbox=dict(facecolor="yellow", alpha=0.5))
    plt.axis("off")
    return fig2img(plt.gcf())

def detect_objects(model_name,url_input,image_input,threshold):

    #Extract model and feature extractor
    feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

    if 'detr' in model_name:

        model = DetrForObjectDetection.from_pretrained(model_name)

    elif 'yolos' in model_name:

        model = YolosForObjectDetection.from_pretrained(model_name)

    if validators.url(url_input):
        image = Image.open(requests.get(url_input, stream=True).raw)

    elif image_input:
        image = image_input

    #Make prediction
    processed_outputs = make_prediction(image, feature_extractor, model)

    #Visualize prediction
    viz_img = visualize_prediction(image, processed_outputs, threshold, model.config.id2label)

    return viz_img

def set_example_image(example: list) -> dict:
    return gr.Image.update(value=example[0])

def set_example_url(example: list) -> dict:
    return gr.Textbox.update(value=example[0])


title = """<h1 id="title">Aplicación de detección de objetos con YOLOS</h1>"""


models = ['hustvl/yolos-small','hustvl/yolos-tiny']
urls = ["https://media.npr.org/assets/img/2023/08/14/defconai-8321_slide-648569f04a3571dc4733dca65d2c0ca3ec672f50.jpg"]


demo = gr.Blocks()

with demo:

    gr.Markdown(title)
    options = gr.Dropdown(choices=models, label='Seleccione el modelo de detección qué desea usar', show_label=True)
    slider_input = gr.Slider(minimum=0.2, maximum=1, value=0.7, label='Umbral de predicción')

    with gr.Tabs():
        with gr.TabItem('Imagen atraves de un URL'):
            with gr.Row():
                url_input = gr.Textbox(lines=2,label='Ingrese una URL valida')
                img_output_from_url = gr.Image() #shape=(650,650)

            with gr.Row():
                example_url = gr.Dataset(components=[url_input],samples=[[str(url)] for url in urls])

            url_but = gr.Button('Detectar')

        with gr.TabItem('Subir imagen'):
            with gr.Row():
                img_input = gr.Image(type='pil')
                img_output_from_upload= gr.Image() #shape=(650,650)

            with gr.Row():
                example_images = gr.Dataset(components=[img_input],
                                            samples=[[path.as_posix()]
                                                     for path in sorted(pathlib.Path('images').rglob('*.JPG'))])

            img_but = gr.Button('Detectar')
        with gr.TabItem('Webcam'):
            with gr.Row():
                webcam_input = gr.Image(type='pil', label='Capturar la imagen desde la webcam')
                webcam_output = gr.Image()

            webcam_but = gr.Button('Detectar desde la Webcam')

    # Tus botones existentes

    # Botón para detectar desde URL
    url_but.click(
        detect_objects,
        inputs=[options, url_input, gr.Image(value=None), slider_input],
        outputs=img_output_from_url,
        queue=True
    )

    # Botón para detectar desde carga de imagen
    img_but.click(
        detect_objects,
        inputs=[options, gr.Textbox(value=""), img_input, slider_input],
        outputs=img_output_from_upload,
        queue=True
    )

    # Botón para detectar desde la webcam
    webcam_but.click(
        detect_objects,
        inputs=[options, gr.Textbox(value=""), webcam_input, slider_input],
        outputs=webcam_output,
        queue=True
    )
    example_images.click(fn=set_example_image,inputs=[example_images],outputs=[img_input])
    example_url.click(fn=set_example_url,inputs=[example_url],outputs=[url_input])

app = demo.launch(share=True)

# La URL pública se imprimirá automáticamente en la salida de Colab
# Pero también puedes obtenerla y guardarla en una variable así
#public_url = app.share_url

# Imprimir la URL pública
#print("Public URL:", public_url)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://564d79b3ee0867dbfc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import qrcode

# Create a QR code object with a larger size and higher error correction
qr = qrcode.QRCode(version=3, box_size=20, border=10, error_correction=qrcode.constants.ERROR_CORRECT_H)

# Define the data to be encoded in the QR code
data = "https://e62cd67e-7e77-491f.gradio.live/"

# Add the data to the QR code object
qr.add_data(data)

# Make the QR code
qr.make(fit=True)

# Create an image from the QR code with a black fill color and white background
img = qr.make_image(fill_color="black", back_color="white")

# Save the QR code image
img.save("qr_code.png")

In [ ]:
!pip freeze > requirements.txt